!pip3 uninstall -y torch torchvision

!pip3 install torch torchvision

In [3]:
import torch
# import the data
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler

In [4]:
import os
os.environ["KERAS_BACKEND"] = "torch"

In [5]:
import numpy as np
import keras
from keras import layers

In [6]:
# function to evaluate model
# returns (accuracy, tpr, tnr, f1)
# takes as input real, predicted
def genResults(real, predicted):
    real=np.argmax(real, axis=1)
    predicted=np.argmax(predicted, axis=1)

    conf_matrix = confusion_matrix(y_true=real, y_pred=predicted)
    tn, fp, fn, tp = conf_matrix.ravel()

    f1 = (2*tp)/((2*tp)+fp+fn)

    acc = (tp + tn) / (tp + tn + fn + fp)

    tpr = tp / (tp + fp)
    tnr = tn / (tn + fn)

    print("true negative: ", tn)
    print("false positive: ", fp)
    print("false negative: ", fn)
    print("true positive: ", tp)
    
    return (acc, tpr, tnr, f1)

In [7]:
# split it like above

# do the shaping if necessary
# change the type of the columns that are characters to one-hot-encodings or something like that

# make a model with one layer
# run the model and see what happens

In [8]:
df = pd.read_csv('data\\online_shoppers_intention.csv')

print(df)

# hyperparameters
batch_size = 128
epochs = 30
dropOut = 0.5

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [9]:
df['total_duration'] = df['Administrative_Duration'] + df['Informational_Duration'] + df['ProductRelated_Duration']
df['proportion_of_duration'] = df['ProductRelated_Duration'] / df['total_duration']

In [10]:
df.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

#df = df.drop(columns=['Informational_Duration', 'Administrative_Duration', 
#                 'ProductRelated_Duration', 'Administrative', 'Informational', 
#                 'ProductRelated', 'OperatingSystems', 'Browser',
#                 'BounceRates', 'ExitRates', 'SpecialDay', 'Month',
#                 'Region', 'TrafficType', 'VisitorType', 'Weekend'])

,BounceRates,ExitRates,PageValues,SpecialDay,Month,Region,TrafficType,VisitorType,Weekend,Revenue,total_duration,proportion_of_duration
0,0.200000,0.200000,0.000000,0.0,Feb,1,1,Returning_Visitor,False,False,0.000000,NaN
1,0.000000,0.100000,0.000000,0.0,Feb,1,2,Returning_Visitor,False,False,64.000000,1.000000
2,0.200000,0.200000,0.000000,0.0,Feb,9,3,Returning_Visitor,False,False,0.000000,NaN
3,0.050000,0.140000,0.000000,0.0,Feb,2,4,Returning_Visitor,False,False,2.666667,1.000000
4,0.020000,0.050000,0.000000,0.0,Feb,1,4,Returning_Visitor,True,False,627.500000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
12325,0.007143,0.029031,12.241717,0.0,Dec,1,1,Returning_Visitor,True,False,1928.791667,0.924823
12326,0.000000,0.021333,0.000000,0.0,Nov,1,8,Returning_Visitor,True,False,465.750000,1.000000
12327,0.083333,0.086667,0.000000,0.0,Nov,1,13,Returning_Visitor,True,False,184.250000,1.000000
12328,0.000000,0.021053,0.000000,0.0,Nov,3,11,Returning_Visitor,False,False,421.000000,0.821853


In [11]:
# df['Month'].replace(['
#print(df['Month'].unique())

In [12]:
df['Month'].replace(['Jan','Feb','Mar','Apr','May','June','Jul','Aug','Sep','Oct','Nov','Dec'],
                    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_15064\727251348.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Month'].replace(['Jan','Feb','Mar','Apr','May','June','Jul','Aug','Sep','Oct','Nov','Dec'],
C:\Users\tomli\AppData\Local\Temp\ipykernel_15064\727251348.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_dow

In [13]:
#print(df['Month'].unique())

In [14]:
#print(df['VisitorType'].unique())

In [15]:
df['VisitorType'].replace(['Returning_Visitor', 'New_Visitor', 'Other'], [0, 1, 2], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_15064\656729720.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['VisitorType'].replace(['Returning_Visitor', 'New_Visitor', 'Other'], [0, 1, 2], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_15064\656729720.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_d

In [16]:
df_copy = df.copy(deep=True)
print (df)

       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

In [17]:
df_copy['Weekend'].replace([False, True], [0,1], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_15064\2402203070.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Weekend'].replace([False, True], [0,1], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_15064\2402203070.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['We

In [18]:
df_copy['Revenue'].replace([False, True], [0,1], inplace=True)

C:\Users\tomli\AppData\Local\Temp\ipykernel_15064\550847130.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy['Revenue'].replace([False, True], [0,1], inplace=True)
C:\Users\tomli\AppData\Local\Temp\ipykernel_15064\550847130.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_copy['Reve

In [19]:
df_copy = df_copy.drop(columns=['Informational_Duration', 'Administrative_Duration', 
                 'ProductRelated_Duration', 'Administrative', 'Informational', 
                 'ProductRelated', 'OperatingSystems', 'Browser'])

In [20]:
df_copy.fillna(0, inplace=True)

In [21]:
df_numpy = df_copy.to_numpy()

In [22]:
df_numpy.shape[0]
testSample = np.random.choice(df_numpy.shape[0], math.floor(0.7 * df_numpy.shape[0]), replace=False)

In [23]:
print(testSample)

[12152  7128 10148 ... 11403   341  3739]


In [24]:
print(df_numpy[testSample])

train = df_numpy[testSample]
test = df_numpy[~testSample]

print("Train length: " + str(train.shape[0]))
print("Test lenght: " + str(test.shape[0]))

[[2.51322800e-03 1.51828800e-02 0.00000000e+00 ... 0.00000000e+00
  6.31806379e+03 9.98417237e-01]
 [1.37681160e-02 3.39070280e-02 0.00000000e+00 ... 0.00000000e+00
  6.97783209e+03 9.66403128e-01]
 [0.00000000e+00 1.73913040e-02 0.00000000e+00 ... 0.00000000e+00
  9.01266667e+02 1.00000000e+00]
 ...
 [0.00000000e+00 1.05263160e-02 0.00000000e+00 ... 0.00000000e+00
  6.86583333e+02 1.00000000e+00]
 [2.00000000e-01 2.00000000e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.42857140e-02 4.04761900e-02 0.00000000e+00 ... 0.00000000e+00
  2.53166667e+02 1.00000000e+00]]
Train length: 8631
Test lenght: 8631


In [25]:
df_copy_y = df_copy['Revenue']
df_copy_x = df_copy.loc[:, df_copy.columns != 'Revenue']

print(df_copy_y)
print(df_copy_x)

0        0
1        0
2        0
3        0
4        0
        ..
12325    0
12326    0
12327    0
12328    0
12329    0
Name: Revenue, Length: 12330, dtype: int64
       BounceRates  ExitRates  PageValues  SpecialDay  Month  Region  \
0         0.200000   0.200000    0.000000         0.0      2       1   
1         0.000000   0.100000    0.000000         0.0      2       1   
2         0.200000   0.200000    0.000000         0.0      2       9   
3         0.050000   0.140000    0.000000         0.0      2       2   
4         0.020000   0.050000    0.000000         0.0      2       1   
...            ...        ...         ...         ...    ...     ...   
12325     0.007143   0.029031   12.241717         0.0     12       1   
12326     0.000000   0.021333    0.000000         0.0     11       1   
12327     0.083333   0.086667    0.000000         0.0     11       1   
12328     0.000000   0.021053    0.000000         0.0     11       3   
12329     0.000000   0.066667    0.000000   

In [26]:
df_copy_x_train, df_copy_x_test, df_copy_y_train, df_copy_y_test = train_test_split(
    df_copy_x, df_copy_y, test_size=0.3, random_state=42)

In [27]:
numpy_x_train = df_copy_x_train.to_numpy()
numpy_x_test = df_copy_x_test.to_numpy()
numpy_y_train = df_copy_y_train.to_numpy()
numpy_y_test = df_copy_y_test.to_numpy()

num_classes = 2

input_shape = (11, 1)

# Make sure images have shape (28, 28, 1)
numpy_x_train = np.expand_dims(numpy_x_train, -1)
numpy_x_test = np.expand_dims(numpy_x_test, -1)
print("x_train shape:", numpy_x_train.shape)
print(numpy_x_train.shape[0], "train samples")
print(numpy_x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
numpy_y_train = keras.utils.to_categorical(numpy_y_train, num_classes)
numpy_y_test = keras.utils.to_categorical(numpy_y_test, num_classes)

print("y_train shape: ", numpy_y_train.shape)

x_train shape: (8631, 11, 1)
8631 train samples
3699 test samples
y_train shape:  (8631, 2)


In [28]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(10, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 11, 10)              │              20 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 110)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 110)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             222 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 242 (968.00 B)

 Trainable params: 242 (968.00 B)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model.fit(numpy_x_train, numpy_y_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3807 - loss: 0.5847 - val_accuracy: 0.8472 - val_loss: 0.1551
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7954 - loss: 0.2013 - val_accuracy: 0.8472 - val_loss: 0.1541
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8126 - loss: 0.1830 - val_accuracy: 0.8472 - val_loss: 0.1533
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8249 - loss: 0.1712 - val_accuracy: 0.8472 - val_loss: 0.1528
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8362 - loss: 0.1592 - val_accuracy: 0.8472 - val_loss: 0.1527
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8497 - loss: 0.1479 - val_accuracy: 0.8472 - val_loss: 0.1525
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8612 - loss: 0.1368 - val_accuracy: 0.8519 - val_loss: 0.1471
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8647 - loss: 0.1331 - val_accuracy: 0.8530 - val_loss:

In [30]:
predicted = model.predict(numpy_x_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [31]:
print("numpy_y_test shape: ", numpy_y_test.shape)
print(numpy_y_test)
print("predicted: ", predicted.shape)
print(predicted)

numpy_y_test shape:  (3699, 2)
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
predicted:  (3699, 2)
[[9.9316943e-01 6.8305763e-03]
 [9.9901068e-01 9.8933838e-04]
 [9.9981552e-01 1.8452099e-04]
 ...
 [9.4781047e-01 5.2189529e-02]
 [2.2971556e-03 9.9770290e-01]
 [9.7772378e-01 2.2276180e-02]]


In [32]:
acc, tpr, tnr, f1 = genResults(numpy_y_test, predicted)

print("10 layers:")
print("Accuracy: ", acc)
print("True-positive rate: ", tpr)
print("True-negative rate: ", tnr)
print("F1 Score: ", f1)

true negative:  3050
false positive:  74
false negative:  367
true positive:  208
10 layers:
Accuracy:  0.8807785888077859
True-positive rate:  0.7375886524822695
True-negative rate:  0.8925958443078724
F1 Score:  0.485414235705951


In [33]:
model20 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model20.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                      │ (None, 11, 20)              │              40 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             442 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 482 (1.88 KB)

 Trainable params: 482 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [34]:
model20.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model20.fit(numpy_x_train, numpy_y_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7850 - loss: 0.2097 - val_accuracy: 0.8461 - val_loss: 0.1542
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8273 - loss: 0.1724 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8358 - loss: 0.1642 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8320 - loss: 0.1677 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8451 - loss: 0.1546 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8417 - loss: 0.1581 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8472 - loss: 0.1527 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8428 - loss: 0.1571 - val_accuracy: 0.8461 - val_loss:

In [35]:
predicted20 = model20.predict(numpy_x_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [36]:
print("numpy_y_test shape: ", numpy_y_test.shape)
print(numpy_y_test)
print("predicted: ", predicted20.shape)
print(predicted)

numpy_y_test shape:  (3699, 2)
[[1. 0.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]]
predicted:  (3699, 2)
[[9.9316943e-01 6.8305763e-03]
 [9.9901068e-01 9.8933838e-04]
 [9.9981552e-01 1.8452099e-04]
 ...
 [9.4781047e-01 5.2189529e-02]
 [2.2971556e-03 9.9770290e-01]
 [9.7772378e-01 2.2276180e-02]]


In [37]:
acc20, tpr20, tnr20, f120 = genResults(numpy_y_test, predicted20)

print("20 layers:")
print("Accuracy: ", acc20)
print("True-positive rate: ", tpr20)
print("True-negative rate: ", tnr20)
print("F1 Score: ", f120)

true negative:  3032
false positive:  92
false negative:  322
true positive:  253
20 layers:
Accuracy:  0.8880778588807786
True-positive rate:  0.7333333333333333
True-negative rate:  0.9039952295766249
F1 Score:  0.55


In [38]:
model40 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(40, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model40.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 11, 40)              │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 440)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 440)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │             882 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 962 (3.76 KB)

 Trainable params: 962 (3.76 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
model40.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model40.fit(numpy_x_train, numpy_y_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6628 - loss: 0.3222 - val_accuracy: 0.8461 - val_loss: 0.1541
Epoch 2/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8331 - loss: 0.1667 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8418 - loss: 0.1575 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8457 - loss: 0.1543 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8517 - loss: 0.1483 - val_accuracy: 0.8461 - val_loss: 0.1540
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8373 - loss: 0.1627 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8456 - loss: 0.1543 - val_accuracy: 0.8461 - val_loss: 0.1539
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8504 - loss: 0.1496 - val_accuracy: 0.8461 - val_loss:

In [40]:
predicted40 = model40.predict(numpy_x_test)

116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [41]:
acc40, tpr40, tnr40, f140 = genResults(numpy_y_test, predicted40)

print("40 layers:")
print("Accuracy: ", acc40)
print("True-positive rate: ", tpr40)
print("True-negative rate: ", tnr40)
print("F1 Score: ", f140)

true negative:  3105
false positive:  19
false negative:  460
true positive:  115
40 layers:
Accuracy:  0.8705055420383887
True-positive rate:  0.8582089552238806
True-negative rate:  0.8709677419354839
F1 Score:  0.3244005641748942


In [42]:
# now we try oversampling
ros = RandomOverSampler(random_state=42)
df_copy_x_resampled, df_copy_y_resampled = ros.fit_resample(df_copy_x, df_copy_y)

df_copy_x_resampled_train, df_copy_x__resampled_test, df_copy_y_resampled_train, df_copy_y_resampled_test = train_test_split(
    df_copy_x_resampled, df_copy_y_resampled, test_size=0.3, random_state=42)

In [43]:
numpy_x_resampled_train = df_copy_x_resampled_train.to_numpy()
numpy_x_resampled_test = df_copy_x__resampled_test.to_numpy()
numpy_y_resampled_train = df_copy_y_resampled_train.to_numpy()
numpy_y_resampled_test = df_copy_y_resampled_test.to_numpy()

num_classes = 2

# input_shape = (2, 1)

# Make sure images have shape (28, 28, 1)
numpy_x_resampled_train = np.expand_dims(numpy_x_resampled_train, -1)
numpy_x_resampled_test = np.expand_dims(numpy_x_resampled_test, -1)
print("x_train shape:", numpy_x_resampled_train.shape)
print(numpy_x_resampled_train.shape[0], "train samples")
print(numpy_x_resampled_test.shape[0], "test samples")

# convert class vectors to binary class matrices
numpy_y_resampled_train = keras.utils.to_categorical(numpy_y_resampled_train, num_classes)
numpy_y_resampled_test = keras.utils.to_categorical(numpy_y_resampled_test, num_classes)

print("y_train shape: ", numpy_y_resampled_train.shape)

x_train shape: (14590, 11, 1)
14590 train samples
6254 test samples
y_train shape:  (14590, 2)


In [44]:
model_r = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(10, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_r.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 11, 10)              │              20 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 110)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 110)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 2)                   │             222 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 242 (968.00 B)

 Trainable params: 242 (968.00 B)

 Non-trainable params: 0 (0.00 B)

In [45]:
model_r.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model_r.fit(numpy_x_resampled_train, numpy_y_resampled_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5261 - loss: 0.4588 - val_accuracy: 0.5291 - val_loss: 0.4656
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6139 - loss: 0.3648 - val_accuracy: 0.7327 - val_loss: 0.2527
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7137 - loss: 0.2562 - val_accuracy: 0.7135 - val_loss: 0.2479
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7415 - loss: 0.2312 - val_accuracy: 0.8245 - val_loss: 0.1515
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7686 - loss: 0.1992 - val_accuracy: 0.8376 - val_loss: 0.1394
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7391 - loss: 0.2288 - val_accuracy: 0.8060 - val_loss: 0.1787
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7900 - loss: 0.1803 - val_accuracy: 0.8273 - val_loss: 0.1427
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7864 - loss: 0.1779 - val_accuracy: 0.

In [46]:
predicted_r = model_r.predict(numpy_x_resampled_test)

196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [47]:
acc_r, tpr_r, tnr_r, f1_r = genResults(numpy_y_resampled_test, predicted_r)

print("10 layers:")
print("Accuracy: ", acc_r)
print("True-positive rate: ", tpr_r)
print("True-negative rate: ", tnr_r)
print("F1 Score: ", f1_r)

true negative:  2668
false positive:  455
false negative:  547
true positive:  2584
10 layers:
Accuracy:  0.8397825391749281
True-positive rate:  0.8502796972688385
True-negative rate:  0.829860031104199
F1 Score:  0.8376012965964343


In [48]:
model_r_20 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_r_20.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                      │ (None, 11, 20)              │              40 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2)                   │             442 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 482 (1.88 KB)

 Trainable params: 482 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
model_r_20.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model_r_20.fit(numpy_x_resampled_train, numpy_y_resampled_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5282 - loss: 0.4633 - val_accuracy: 0.5469 - val_loss: 0.4420
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5971 - loss: 0.3953 - val_accuracy: 0.6923 - val_loss: 0.2917
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6465 - loss: 0.3462 - val_accuracy: 0.6347 - val_loss: 0.3537
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6676 - loss: 0.3239 - val_accuracy: 0.8245 - val_loss: 0.1653
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7019 - loss: 0.2924 - val_accuracy: 0.8245 - val_loss: 0.1677
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7324 - loss: 0.2608 - val_accuracy: 0.8266 - val_loss: 0.1668
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7372 - loss: 0.2556 - val_accuracy: 0.7779 - val_loss: 0.2161
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7689 - loss: 0.2244 - val_accuracy: 0.

In [50]:
predicted_r_20 = model_r_20.predict(numpy_x_resampled_test)

196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [51]:
acc_r_20, tpr_r_20, tnr_r_20, f1_r_20 = genResults(numpy_y_resampled_test, predicted_r_20)

print("20 layers:")
print("Accuracy: ", acc_r_20)
print("True-positive rate: ", tpr_r_20)
print("True-negative rate: ", tnr_r_20)
print("F1 Score: ", f1_r_20)

true negative:  2828
false positive:  295
false negative:  814
true positive:  2317
20 layers:
Accuracy:  0.822673488967061
True-positive rate:  0.8870597243491577
True-negative rate:  0.7764964305326744
F1 Score:  0.8068953508619189


In [52]:
model_r_40 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(40, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_r_40.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 11, 40)              │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_5 (Flatten)                  │ (None, 440)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 440)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 2)                   │             882 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 962 (3.76 KB)

 Trainable params: 962 (3.76 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
model_r_40.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model_r_40.fit(numpy_x_resampled_train, numpy_y_resampled_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5371 - loss: 0.4503 - val_accuracy: 0.5990 - val_loss: 0.3891
Epoch 2/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5881 - loss: 0.4053 - val_accuracy: 0.5942 - val_loss: 0.4001
Epoch 3/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6213 - loss: 0.3743 - val_accuracy: 0.6127 - val_loss: 0.3843
Epoch 4/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6366 - loss: 0.3577 - val_accuracy: 0.6552 - val_loss: 0.3391
Epoch 5/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6717 - loss: 0.3238 - val_accuracy: 0.7012 - val_loss: 0.2913
Epoch 6/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6725 - loss: 0.3248 - val_accuracy: 0.7080 - val_loss: 0.2872
Epoch 7/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6890 - loss: 0.3063 - val_accuracy: 0.7382 - val_loss: 0.2569
Epoch 8/30
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7035 - loss: 0.2925 - val_accuracy: 0.

In [54]:
predicted_r_40 = model_r_40.predict(numpy_x_resampled_test)

196/196 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [55]:
acc_r_40, tpr_r_40, tnr_r_40, f1_r_40 = genResults(numpy_y_resampled_test, predicted_r_40)

print("40 layers:")
print("Accuracy: ", acc_r_40)
print("True-positive rate: ", tpr_r_40)
print("True-negative rate: ", tnr_r_40)
print("F1 Score: ", f1_r_40)

true negative:  2619
false positive:  504
false negative:  569
true positive:  2562
40 layers:
Accuracy:  0.8284298049248481
True-positive rate:  0.8356164383561644
True-negative rate:  0.8215181932245922
F1 Score:  0.826851702436663


In [56]:
# Now we need to repeat it all - starting with 20 nodes I think, but with a much smaller fraction
small_x_resampled_train, small_x_resampled_test, small_y_resampled_train, small_y_resampled_test = train_test_split(
    df_copy_x_resampled, df_copy_y_resampled, test_size=0.7, random_state=42)

numpy_small_x_train = small_x_resampled_train.to_numpy()
numpy_small_x_test = small_x_resampled_test.to_numpy()
numpy_small_y_train = small_y_resampled_train.to_numpy()
numpy_small_y_test = small_y_resampled_test.to_numpy()

num_classes = 2

# input_shape = (2, 1)

# Make sure images have shape (11, 1)
numpy_small_x_train = np.expand_dims(numpy_small_x_train, -1)
numpy_small_x_test = np.expand_dims(numpy_small_x_test, -1)
print("x_train shape:", numpy_small_x_train.shape)
print(numpy_small_x_train.shape[0], "train samples")
print(numpy_small_x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
numpy_small_y_train = keras.utils.to_categorical(numpy_small_y_train, num_classes)
numpy_small_y_test = keras.utils.to_categorical(numpy_small_y_test, num_classes)

print("y_train shape: ", numpy_small_y_train.shape)

x_train shape: (6253, 11, 1)
6253 train samples
14591 test samples
y_train shape:  (6253, 2)


In [57]:
model_small_r_20 = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(20, activation="relu"),
        layers.Flatten(),
        layers.Dropout(dropOut),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_small_r_20.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 11, 20)              │              40 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 220)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 2)                   │             442 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 482 (1.88 KB)

 Trainable params: 482 (1.88 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
model_small_r_20.compile(loss="mean_squared_error", optimizer="adam", 
              metrics=["accuracy"])

model_small_r_20.fit(numpy_small_x_train, numpy_small_y_train, batch_size=batch_size, 
          epochs=epochs, validation_split=0.1)

Epoch 1/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5357 - loss: 0.4582 - val_accuracy: 0.6086 - val_loss: 0.3787
Epoch 2/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5388 - loss: 0.4547 - val_accuracy: 0.5735 - val_loss: 0.4246
Epoch 3/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5295 - loss: 0.4650 - val_accuracy: 0.5815 - val_loss: 0.4164
Epoch 4/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5526 - loss: 0.4428 - val_accuracy: 0.6182 - val_loss: 0.3773
Epoch 5/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5593 - loss: 0.4346 - val_accuracy: 0.6374 - val_loss: 0.3555
Epoch 6/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5750 - loss: 0.4201 - val_accuracy: 0.5974 - val_loss: 0.4008
Epoch 7/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5764 - loss: 0.4192 - val_accuracy: 0.5735 - val_loss: 0.4256
Epoch 8/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5470 - loss: 0.4492 - val_accuracy: 0.5879 - val_loss:

In [59]:
predicted_r_small_20 = model_small_r_20.predict(numpy_small_x_test)

456/456 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [60]:
print(small_y_resampled_test.shape)
print(predicted_r_small_20.shape)

(14591,)
(14591, 2)


In [61]:
acc_r_small_20, tpr_r_small_20, tnr_r_small_20, f1_r_small_20 = genResults(numpy_small_y_test, predicted_r_small_20)

print("Small resampled 20 layers:")
print("Accuracy: ", acc_r_small_20)
print("True-positive rate: ", tpr_r_small_20)
print("True-negative rate: ", tnr_r_small_20)
print("F1 Score: ", f1_r_small_20)

true negative:  6386
false positive:  925
false negative:  2215
true positive:  5065
Small resampled 20 layers:
Accuracy:  0.7847988486053047
True-positive rate:  0.845575959933222
True-negative rate:  0.7424718056039995
F1 Score:  0.7633760361718162
